Demonstrating my function to compare ranked lists
-------------

A notebook to demonstrate [a function for comparing ranked lists](https://github.com/fomightez/text_mining), called `function to generate accounting of positions in two ranked lists.py`.



Provided input files:  
   Two similar, ranked list of yeast genes:

   * original_ranked_list.txt

   * similar_gene_list.txt

All other code is foud in the notebook.
    
------
